In [6]:
import SpecialPolynomials as sp
using BenchmarkTools: @btime

In [7]:
H = [sp.basis(sp.Hermite, i) for i in 0:30]

n = 20 + 1
H[n + 1](3.0)

4.7763998631936e13

In [8]:
@btime H[n](3.0)

  63.849 ns (1 allocation: 16 bytes)


5.9990281399296e13

In [11]:
test = sp.basis(sp.Hermite, 20)

SpecialPolynomials.Hermite(1.0⋅H₂₀(x))

In [12]:
@btime test(3.0)

  52.482 ns (1 allocation: 16 bytes)


5.9990281399296e13

In [ ]:
struct Hermite{N} end
Hermite(n) = Hermite{n}()

In [ ]:
H = Hermite(3)

In [ ]:
for n in 0:20
    ex = "0 "
    for m in 0:n÷2
        factor = factorial(n) * (-1)^m / (factorial(m) * factorial(n - 2m)) * 2^(n - 2m)
        exponent = n - 2m

        ex *= "+ $factor * x^$exponent "
    end
    println("n = $n : $ex")
    @eval $(Meta.parse("global hermite(x, H::Hermite{$(n)}) = $ex"))
end

In [ ]:
@generated function hermite(x, H::Hermite{N}) where N
    ex = "0 "
    n = N
    for m in 0:n÷2
        factor = factorial(n) * (-1)^m / (factorial(m) * factorial(n - 2m)) * 2^(n - 2m)
        exponent = n - 2m

        ex *= "+ $factor * x^$exponent "
    end
    print(ex)
    return Meta.parse(ex)
end

In [ ]:
@generated function hermite(x, H::Hermite{N}) where N
    ex = :(0)
    n = N
    for m in 0:n÷2
        factor = factorial(n) * (-1)^m / (factorial(m) * factorial(n - 2m)) * 2^(n - 2m)
        exponent = n - 2m

        ex = :($ex + $factor * x^$exponent)
    end
    return ex
end

In [ ]:
@generated function hermite(x, H::Hermite{N}) where N
    return :(8x^3 - 12x)
end

In [ ]:
@time hermite(3.0, H)

In [ ]:
n = 3
ex = "0 "
for m in 0:n÷2
    factor = factorial(n) * (-1)^m / (factorial(m) * factorial(n - 2m)) * 2^(n - 2m)
    exponent = n - 2m

    ex *= "+ $factor * x^$exponent "
end
println("n = $n : $ex")
@eval $(Meta.parse("hermite(x, H::Hermite{$(n)}) = $ex"))

In [ ]:
hermite

In [ ]:
@btime H3(3.0)

In [ ]:
@btime H(3.0)

In [ ]:
function H3(x)
    return 8.0x^3 - 12.0x
end

In [ ]:
H4(x) = 8.0x^3 - 12.0x

In [ ]:
function slow()
    H4(3.0)
end

In [ ]:
@btime slow()

In [ ]:
function timeit(func, x)
    func(x)
end

In [ ]:
@time timeit(H, 3.0)

In [ ]:
function H2(x)
    8x^3 - 12x
end

In [ ]:
@btime H2(3.0)

In [ ]:
function hermite(n, x)
    result = 0.0
    
    for m in 0:n÷2
        factor = factorial(n) * (-1)^m / (factorial(m) * factorial(n - 2m)) * 2^(n - 2m)
        exponent = n - 2m

        result += factor * x^exponent
    end
    return result
end

In [ ]:
@btime hermite(9, 3.0)

In [ ]:
funcs = Function[print, sum]

In [ ]:
funcs = Function[]
N = 20
for n in 0:N
    ex = "0"
    for m in 0:n÷2
        factor = factorial(n) * (-1)^m / (factorial(m) * factorial(n - 2m)) * 2^(n - 2m)
        exponent = n - 2m

        ex *= "+ $factor * x^$exponent "
    end

    push!(funcs, @eval $(Meta.parse("x -> $ex")))
end

In [ ]:
@btime funcs[21](3.0)

In [ ]:
struct Hermite{T}
    hermite::Array{T}
    function Hermite(N)
        funcs = Function[]
        for n in 0:N
            ex = "0"
            for m in 0:n÷2
                factor = (-1)^m / (factorial(m) * factorial(n - 2m)) * 2^(n - 2m)
                exponent = n - 2m

                ex *= "+ $factor * x^$exponent"
            end
            
            push!(funcs, @eval $(Meta.parse("x -> factorial($n) * $ex")))
        end
        return new{typeof(hermite)}(hermite)
    end
end

In [ ]:
H = Hermite(20)

In [ ]:
@btime H.hermite(3.0)

In [ ]:
21%2